In [1024]:
import numpy as np 
from tqdm import tqdm
import pandas as pd
import random
import ast
import time

In [1025]:
df = pd.read_csv("train.csv")
df

,untagged_sentence,tagged_sentence
0,"['The', 'jury', 'further', 'said', 'in', 'term...","[('The', 'AT'), ('jury', 'NN'), ('further', 'R..."
1,"['The', 'September-October', 'term', 'jury', '...","[('The', 'AT'), ('September-October', 'NP'), (..."
2,"['``', 'Only', 'a', 'relative', 'handful', 'of...","[('``', '``'), ('Only', 'RB'), ('a', 'AT'), ('..."
3,"['The', 'jury', 'said', 'it', 'did', 'find', '...","[('The', 'AT'), ('jury', 'NN'), ('said', 'VB')..."
4,"['It', 'recommended', 'that', 'Fulton', 'legis...","[('It', 'PP'), ('recommended', 'VB'), ('that',..."
...,...,...
47335,"['S.', 'J.', 'Perelman']","[('S.', 'NP'), ('J.', 'NP'), ('Perelman', 'NP')]"
47336,"['revulsion', 'in', 'the', 'desert']","[('revulsion', 'NN'), ('in', 'IN'), ('the', 'A..."
47337,"['the', 'doors', 'of', 'the', 'D', 'train', 's...","[('the', 'AT'), ('doors', 'NN'), ('of', 'IN'),..."
47338,"['She', 'was', 'a', 'living', 'doll', 'and', '...","[('She', 'PP'), ('was', 'BE'), ('a', 'AT'), ('..."


In [1026]:
data = []
for index, row in tqdm(df.iterrows()):
    data.append(ast.literal_eval(row['tagged_sentence']))

47340it [00:06, 7472.15it/s] 


In [1027]:
x = []
for i in range(len(data)):
    for j in range(len(data[i])):
        if data[i][j][1] not in x:
            x.append(data[i][j][1])
print(len(x))

49


In [1028]:
count_of_tags = {string: 0 for i, string in enumerate(x)}
for i in range(len(data)):
    for j in range(len(data[i])):
        count_of_tags[data[i][j][1]] += 1

In [1029]:
row_names = ['start'] + x + ['final']
col_names = x
some_matrix = [[0 for _ in range(49)] for _ in range(51)]
trans_prob_mat = pd.DataFrame(some_matrix, index=row_names, columns=col_names)

In [1030]:
for i in range(len(data)):
    trans_prob_mat[data[i][0][1]]['start'] += 1
for i in range(len(data)):
    for j in range(len(data[i])-1):
        trans_prob_mat[data[i][j+1][1]][data[i][j][1]] += 1
for i in range(len(data)):
    trans_prob_mat[data[i][len(data[i])-1][1]]['final'] += 1

In [1031]:
#Calculation of the Transition Probability Matrix (A)
for i in range(49):
    trans_prob_mat[x[i]]['start'] = (trans_prob_mat[x[i]]['start'] + 1) / (47340 + 51)
    trans_prob_mat[x[i]]['final'] = (trans_prob_mat[x[i]]['final'] + 1) / (47340 + 51)

for i in range(49):
    for ele in col_names:
        trans_prob_mat[x[i]][ele] = (trans_prob_mat[x[i]][ele] + 1) / (count_of_tags[ele] + 49) #Laplacian smoothing
        
trans_prob_mat

/var/folders/02/r7s9rg8j7nd6qr8phf1lwd600000gn/T/ipykernel_77298/1479961985.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans_prob_mat[x[i]]['start'] = (trans_prob_mat[x[i]]['start'] + 1) / (47340 + 51)


,AT,NN,RB,VB,IN,CS,JJ,",",WD,HV,...,)-,',FW,UH,:-,.-,*-,WQ,RN,NI
start,0.146104,0.060222,0.066848,0.035112,0.083771,0.038699,0.025174,0.000190,0.006309,0.001625,...,0.000021,0.000317,0.000506,0.002511,0.000021,0.000422,0.000063,0.000739,0.000021,0.000084
AT,0.000024,0.608541,0.004039,0.036366,0.000061,0.000012,0.228589,0.000147,0.000037,0.000012,...,0.000037,0.000282,0.001750,0.000037,0.000012,0.000012,0.000012,0.000012,0.000012,0.000024
NN,0.006716,0.103319,0.019189,0.064835,0.260630,0.019950,0.009379,0.121549,0.006609,0.012297,...,0.000064,0.000389,0.000144,0.000027,0.000362,0.000927,0.000005,0.000069,0.000005,0.000064
RB,0.055426,0.015781,0.028176,0.221001,0.145322,0.049676,0.042009,0.130307,0.002971,0.010414,...,0.000032,0.000192,0.000128,0.000096,0.000096,0.000128,0.000032,0.000192,0.000032,0.000032
VB,0.131038,0.110329,0.050610,0.014116,0.206114,0.038018,0.033928,0.045968,0.003985,0.000636,...,0.000031,0.000303,0.000271,0.000156,0.000188,0.000188,0.000010,0.000334,0.000021,0.000010
IN,0.356997,0.201985,0.008461,0.037306,0.010964,0.001316,0.074582,0.001385,0.011746,0.000683,...,0.000020,0.000089,0.000495,0.000049,0.000010,0.000040,0.000010,0.000158,0.000059,0.000010
CS,0.254392,0.077695,0.018272,0.037953,0.035133,0.031013,0.042073,0.009922,0.002657,0.001681,...,0.000054,0.000325,0.000325,0.000054,0.000054,0.000054,0.000054,0.000054,0.000054,0.000054
JJ,0.003491,0.635018,0.004705,0.010911,0.055282,0.024639,0.049160,0.050796,0.000304,0.000658,...,0.000084,0.000422,0.000540,0.000051,0.000017,0.000236,0.000017,0.000017,0.000017,0.000034
",",0.092833,0.067591,0.058684,0.107358,0.079411,0.051296,0.045740,0.001290,0.013214,0.008761,...,0.000021,0.000645,0.001061,0.001457,0.000021,0.000021,0.000021,0.000541,0.000021,0.000146
WD,0.086248,0.073160,0.030251,0.175713,0.021026,0.001931,0.017164,0.022098,0.000215,0.045269,...,0.000215,0.000215,0.000429,0.000215,0.000215,0.000215,0.000215,0.000215,0.000215,0.000215


In [1032]:
arr = np.array(trans_prob_mat)
len(arr[50])

49

In [1033]:
vocab = set()
for row in data:
    for word in row:
        vocab.add(word[0])
vocab = list(vocab)
print(len(vocab))

51208


In [1034]:
row_names = x
col_names = vocab
matrix = [[0 for _ in range(len(vocab))] for _ in range(len(x))]
emiss_prob_mat = pd.DataFrame(matrix, index=row_names, columns=col_names)

In [1035]:
for i in range(len(data)):
    for j in range(len(data[i])):
        emiss_prob_mat[data[i][j][0]][data[i][j][1]] += 1

In [1036]:
for i in range(len(vocab)):
    for ele in row_names:
        emiss_prob_mat[vocab[i]][ele] = (emiss_prob_mat[vocab[i]][ele] + 1) / (count_of_tags[ele] + len(vocab))

/var/folders/02/r7s9rg8j7nd6qr8phf1lwd600000gn/T/ipykernel_77298/4240604834.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emiss_prob_mat[vocab[i]][ele] = (emiss_prob_mat[vocab[i]][ele] + 1) / (count_of_tags[ele] + len(vocab))


In [1037]:
emiss_prob_mat

,dentist,Fashions,I.M.F.,whitehaired,bricks,1900,reminisced,unavailable,Tris,roundness,...,feathered,chump,superfluous,Displaying,transplantable,smoothness,gear,melting,Sword,minds
AT,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
NN,0.000046,0.000008,0.000004,0.000004,0.000025,0.000004,0.000004,0.000004,0.000004,0.000008,...,0.000004,0.000008,0.000004,0.000004,0.000004,0.000021,0.000092,0.000013,0.000008,0.000197
RB,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,...,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012
VB,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000014,0.000007,0.000007,0.000007,...,0.000034,0.000007,0.000007,0.000014,0.000007,0.000007,0.000007,0.000109,0.000007,0.000007
IN,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,...,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007
CS,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,...,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014
JJ,0.000009,0.000009,0.000009,0.000018,0.000009,0.000009,0.000009,0.000063,0.000009,0.000009,...,0.000009,0.000009,0.000036,0.000009,0.000018,0.000009,0.000009,0.000018,0.000009,0.000009
",",0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,...,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
WD,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,...,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018
HV,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,...,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016


In [1038]:
test = pd.read_csv("test_small.csv")
test

,Unnamed: 0,id,untagged_sentence
0,3,3,"['For', 'you', 'have', 'been', 'reborn', ',', ..."
1,4,4,"['Man', 'had', 'enough', 'to', 'worry', 'about..."
2,6,6,"['He', 'trudged', 'on', ',', 'his', 'aching', ..."
3,12,12,"['``', 'But', 'I', 'believe', 'if', 'people', ..."
4,15,15,"['then', ',', 'to', 'better', 'purpose', ',', ..."
...,...,...,...
3995,9990,9990,"['The', 'way', 'MacArthur', 'said', 'his', 'li..."
3996,9991,9991,"['urged', 'the', 'reconvention', 'of', 'the', ..."
3997,9994,9994,"['A', 'prospective', 'industry', 'also', 'may'..."
3998,9997,9997,"['Mr.', 'McCormack', '.']"


In [1039]:
test_data = []
for index, row in tqdm(test.iterrows()):
    test_data.append(ast.literal_eval(row['untagged_sentence']))

4000it [00:00, 18438.28it/s]


In [1040]:
y = x
x = ["start"] + x + ["final"]
vocab = set(vocab)

In [1041]:
cnt = 0
all_tags = [ele for ele in y]
values = [0] * len(all_tags) 
pos_tags_dict = {tag: value for tag, value in zip(all_tags, values)}
for i in range(len(data)):
    for j in range(len(data[i])):
            pos_tags_dict[data[i][j][1]] += 1
            cnt += 1
for tag in pos_tags_dict:
    pos_tags_dict[tag] /= cnt
    pos_tags_dict[tag] *= 100
    
choices = []
for tag in pos_tags_dict:
    for i in range(int(pos_tags_dict[tag])):
        choices.append(tag)

In [1053]:
x

['start',
 'AT',
 'NN',
 'RB',
 'VB',
 'IN',
 'CS',
 'JJ',
 ',',
 'WD',
 'HV',
 '``',
 'CC',
 'NP',
 "''",
 'BE',
 '.',
 'TO',
 'DT',
 'PP',
 'DO',
 'AP',
 'QL',
 'AB',
 'WR',
 'CD',
 'MD',
 'PN',
 'WP',
 '*',
 'EX',
 ':',
 '(',
 ')',
 'NR',
 'RP',
 '--',
 'OD',
 ',-',
 '(-',
 ')-',
 "'",
 'FW',
 'UH',
 ':-',
 '.-',
 '*-',
 'WQ',
 'RN',
 'NI',
 'final']

In [1043]:
all_tags = [ele for ele in y]
values = [i for i in range(49)]
tags_to_index = {tag: value for tag , value in zip(all_tags,values)}
tags_to_index

{'AT': 0,
 'NN': 1,
 'RB': 2,
 'VB': 3,
 'IN': 4,
 'CS': 5,
 'JJ': 6,
 ',': 7,
 'WD': 8,
 'HV': 9,
 '``': 10,
 'CC': 11,
 'NP': 12,
 "''": 13,
 'BE': 14,
 '.': 15,
 'TO': 16,
 'DT': 17,
 'PP': 18,
 'DO': 19,
 'AP': 20,
 'QL': 21,
 'AB': 22,
 'WR': 23,
 'CD': 24,
 'MD': 25,
 'PN': 26,
 'WP': 27,
 '*': 28,
 'EX': 29,
 ':': 30,
 '(': 31,
 ')': 32,
 'NR': 33,
 'RP': 34,
 '--': 35,
 'OD': 36,
 ',-': 37,
 '(-': 38,
 ')-': 39,
 "'": 40,
 'FW': 41,
 'UH': 42,
 ':-': 43,
 '.-': 44,
 '*-': 45,
 'WQ': 46,
 'RN': 47,
 'NI': 48}

In [768]:
#Submission 1
all_predictions = []
for i in range(len(test_data)):
    path_prob_mat = [[0 for _ in range(len(test_data[i]))] for _ in range(49)]
    for j in range(49):
        if test_data[i][0] not in vocab:
            path_prob_mat[1][0] = 1
        else:
            path_prob_mat[j][0] = trans_prob_mat[y[j]]['start'] * emiss_prob_mat[test_data[i][0]][y[j]]
    print(path_prob_mat)
        
    for k in range(1,len(test_data[i])):
        for j in range(49):
            for l in range(49):
                if test_data[i][k] not in vocab:
                    path_prob_mat[1][k] = 1
                else:
                    path_prob_mat[j][k] += path_prob_mat[l][k-1] * trans_prob_mat[y[j]][y[l]] * emiss_prob_mat[test_data[i][k]][y[j]]
    d = np.array(path_prob_mat)   
    preds = []
    for j in range(len(test_data[i])):
        preds.append(y[np.argmax(d[:,j])])
    all_predictions.append(preds)
print("Execution done")

[[1.0997146577810535e-06, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.520535774442168e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8.106538250584818e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.388435544516976e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.00048265179100435214, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.00010730817495866546, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.2790780100903e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.91415919771478e-09, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.130278543500539e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.6589550560922348e-08, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.2427450231999287e-06, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.957794270959514e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8.6414807631

[[1.0997146577810535e-06, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.520535774442168e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8.106538250584818e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.388435544516976e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.503441174508007e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.560009065215827e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.2790780100903e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.91415919771478e-09, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.130278543500539e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.6589550560922348e-08, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.2427450231999287e-06, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.957794270959514e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8.641480763173678e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.7439600705516037e-08, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [4.204316222302103e-08, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8.334325217004269e-09, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [6.770923791236388e-08, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [6.605639747305911e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.00881

[[1.0997146577810535e-06, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.520535774442168e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [6.079903687938613e-05, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.388435544516976e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.503441174508007e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.560009065215827e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.2790780100903e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.91415919771478e-09, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.130278543500539e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.6589550560922348e-08, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.2427450231999287e-06, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.957794270959514e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8.641480763173678e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.74396007

[[1.0997146577810535e-06, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.520535774442168e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.0004563981035079252, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.388435544516976e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.503441174508007e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.560009065215827e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3.87443261715351e-06, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.91415919771478e-09, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.130278543500539e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.6589550560922348

[[1.0997146577810535e-06, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.520535774442168e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8.106538250584818e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.388435544516976e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.503441174508007e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.560009065215827e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.2790780100903e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.91415919771478e-09, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.130278543500539e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.6589550560922348e-08, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.018110523223092558, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.957794270959514e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8.641480763173678e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.7439600705516037e-08, 0, 0, 0, 0, 0, 0, 0, 0, 0], [4.204316222302103e-08, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8.334325217004269e-09, 0, 0, 0, 0, 0, 0, 0, 0, 0], [6.770923791236388e-08, 0, 0, 0, 0, 0, 0, 0, 0, 0], [6.605639747305911e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.7796264022513968e-06, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.04438558

...

[[1.0997146577810535e-06, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.520535774442168e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8.106538250584818e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.388435544516976e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.6510323523524021e-06, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.560009065215827e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.2790780100903e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.91415919771478e-09, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.130278543500539e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.6589550560922348e-08, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.2427450231999287e-06, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.957794270959514e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8.641480763173678e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.7439600705516037e-08, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [4.204316222302103e-08, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8.334325217004269e-09, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [6.770923791236388e-08, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [6.605639747305911e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.7796

[[0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
[[1.099714657781053

[[1.0997146577810535e-06, 0, 0, 0], [2.520535774442168e-07, 0, 0, 0], [8.106538250584818e-07, 0, 0, 0], [2.388435544516976e-07, 0, 0, 0], [5.503441174508007e-07, 0, 0, 0], [5.560009065215827e-07, 0, 0, 0], [2.2790780100903e-07, 0, 0, 0], [1.91415919771478e-09, 0, 0, 0], [1.130278543500539e-07, 0, 0, 0], [2.6589550560922348e-08, 0, 0, 0], [1.2427450231999287e-06, 0, 0, 0], [5.957794270959514e-07, 0, 0, 0], [8.641480763173678e-07, 0, 0, 0], [2.7439600705516037e-08, 0, 0, 0], [4.204316222302103e-08, 0, 0, 0], [8.334325217004269e-09, 0, 0, 0], [6.770923791236388e-08, 0, 0, 0], [6.605639747305911e-07, 0, 0, 0], [0.0015571731019699722, 0, 0, 0], [5.044385587588154e-08, 0, 0, 0], [1.4634422765773213e-07, 0, 0, 0], [8.669201771484115e-08, 0, 0, 0], [1.077135862662685e-07, 0, 0, 0], [2.7455329840437854e-07, 0, 0, 0], [2.647067615757064e-07, 0, 0, 0], [4.334951331565748e-08, 0, 0, 0], [9.71816194979835e-08, 0, 0, 0], [2.2266188859591127e-08, 0, 0, 0], [5.026069921584385e-08, 0, 0, 0], [2.7634271

[[1.0997146577810535e-06, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.520535774442168e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8.106538250584818e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.388435544516976e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.503441174508007e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.560009065215827e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.2790780100903e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.91415919771478e-09, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1.130278543500539e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.6589550560922348e-08, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.018110523223092558, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5.957794270959514e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8.641480763173678e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2.7439600705

KeyboardInterrupt: 

In [725]:
all_predictions[0]

['IN',
 'PP',
 'HV',
 'BE',
 'VB',
 ',',
 '*',
 'IN',
 'JJ',
 'NN',
 'CC',
 'IN',
 'AT',
 ',',
 'IN',
 'AT',
 'NN',
 'IN',
 'NP',
 '.']

In [567]:
#Submission 2
all_predictions = []
for i in range(len(test_data)):
    path_prob_mat = [[0 for _ in range(len(test_data[i]))] for _ in range(49)]
    for j in range(49):
        if test_data[i][0] not in vocab:
            path_prob_mat[1][0] = 1
        else:
            path_prob_mat[j][0] = trans_prob_mat[y[j]]['start'] * emiss_prob_mat[test_data[i][0]][y[j]]
        
    for k in range(1,len(test_data[i])):
        for j in range(49):
            for l in range(49):
                if test_data[i][k] not in vocab:
                    #path_prob_mat[1][k] = 1
                    maxi = -1
                    index = -1
                    for m in range(49):
                        if path_prob_mat[m][k-1] > maxi:
                            maxi = path_prob_mat[m][k-1]
                            index = m
                    path_prob_mat[np.argmax(arr[index])][k] = 1
                else:
                    path_prob_mat[j][k] += path_prob_mat[l][k-1] * trans_prob_mat[y[j]][y[l]] * emiss_prob_mat[test_data[i][k]][y[j]]
    d = np.array(path_prob_mat)   
    preds = []
    for j in range(len(test_data[i])):
        preds.append(y[np.argmax(d[:,j])])
    all_predictions.append(preds)
print("Execution done")

Execution done


In [643]:
#Submission 3
all_predictions = []
for i in range(len(test_data)):
    path_prob_mat = [[0 for _ in range(len(test_data[i]))] for _ in range(49)]
    for j in range(49):
        if test_data[i][0] not in vocab:
            path_prob_mat[tags_to_index[random.choice(choices)]][0] = 1
        else:
            path_prob_mat[j][0] = trans_prob_mat[y[j]]['start'] * emiss_prob_mat[test_data[i][0]][y[j]]
        
    for k in range(1,len(test_data[i])):
        for j in range(49):
            for l in range(49):
                if test_data[i][k] not in vocab:
                    path_prob_mat[tags_to_index[random.choice(choices)]][k] = 1
                else:
                    path_prob_mat[j][k] += path_prob_mat[l][k-1] * trans_prob_mat[y[j]][y[l]] * emiss_prob_mat[test_data[i][k]][y[j]]
    d = np.array(path_prob_mat)   
    preds = []
    for j in range(len(test_data[i])):
        preds.append(y[np.argmax(d[:,j])])
    all_predictions.append(preds)
print("Execution done")

Execution done


In [913]:
#Actual Viterbi
all_predictions = []
for i in range(len(test_data)):
    path_prob_mat = [[0 for _ in range(len(test_data[i]))] for _ in range(50)]
    backpointer = [[0 for _ in range(len(test_data[i]))] for _ in range(50)]
    for j in range(49):
        if data[i][0] not in vocab:
            path_prob_mat[1][0] = 1
        else:
            path_prob_mat[j][0] = trans_prob_mat[y[j]]['start'] * emiss_prob_mat[test_data[i][0]][y[j]]

    for k in range(1,len(test_data[i])):
        if test_data[i][k] not in vocab:
            path_prob_mat[1][k] = 1
            backpointer[1][k] = 1
            continue
        for j in range(49):
            temp1 = []
            temp2 = []
            for l in range(49):
                temp = path_prob_mat[l][k-1] * trans_prob_mat[y[j]][y[l]]
                temp1.append(temp * emiss_prob_mat[test_data[i][k]][y[j]])
                temp2.append(temp)
            numpy_temp1 = np.array(temp1)
            numpy_temp2 = np.array(temp2)
            path_prob_mat[j][k] = np.max(numpy_temp1)
            backpointer[j][k] = np.argmax(numpy_temp2)
            
    temp = []
    for j in range(49):
        temp.append(path_prob_mat[j][len(test_data[i])-1] * trans_prob_mat[y[j]]['final'])
    numpy_temp = np.array(temp)
    path_prob_mat[49][len(test_data[i])-1] = np.max(numpy_temp)
    backpointer[49][len(test_data[i]) - 1] = np.argmax(numpy_temp)
    
    preds = []
    temp = backpointer[49][len(test_data[i])-1]
    preds.append(y[temp])
    for j in range(len(test_data[i])-1,0,-1):
        preds.append(y[backpointer[temp][j]])
        temp = backpointer[temp][j]
    preds.reverse()
    all_predictions.append(preds)

In [1055]:
trans_prob_mat(['start'][15])

IndexError: list index out of range

In [1049]:
#Actual Viterbi testing
all_predictions = []
for i in range(50,51):
    path_prob_mat = [[0 for _ in range(len(data[i]))] for _ in range(50)]
    backpointer = [[0 for _ in range(len(data[i]))] for _ in range(50)]
    if data[i][0][0] not in vocab:
            path_prob_mat[1][0] = 1
            #backpointer[1][0] = 1
    else:
        for j in range(49):
            path_prob_mat[j][0] = trans_prob_mat[y[j]]['start'] * emiss_prob_mat[data[i][0][0]][y[j]]

    for k in range(1,len(data[i])):
        if data[i][k][0] not in vocab:
            path_prob_mat[1][k] = 1
            backpointer[1][k] = 1
            continue
            
        for j in range(49):
            temp1 = []
            temp2 = []
            for l in range(49):
                temp = path_prob_mat[l][k-1] * trans_prob_mat[y[j]][y[l]]
                temp1.append(temp * emiss_prob_mat[data[i][k][0]][y[j]])
                temp2.append(temp)
            path_prob_mat[j][k] = np.max(np.array(temp1))
            backpointer[j][k] = np.argmax(np.array(temp2))
            
    temp = []
    for j in range(49):
        temp.append(path_prob_mat[j][len(data[i])-1] * trans_prob_mat[y[j]]['final'])
    path_prob_mat[49][len(data[i])-1] = np.max(np.array(temp))
    backpointer[49][len(data[i]) - 1] = np.argmax(np.array(temp))
    
    preds = []
    temp = backpointer[49][len(data[i])-1]
    preds.append(y[temp])
    for j in range(len(data[i])-1,0,-1):
        preds.append(y[backpointer[temp][j]])
        temp = backpointer[temp][j]
    preds.reverse()
    all_predictions.append(preds)

In [1050]:
print(path_prob_mat[0])

[1.0997146577810535e-06, 9.106665529912231e-06, 1.6780391923541555e-15, 3.9617062412320675e-18, 1.2153533681939563e-21, 1.9871377761253352e-25, 4.2725451054206606e-27, 9.249841379450395e-33, 8.038687094107372e-35, 2.8115898350988744e-38, 9.415756759845912e-40, 6.512224094821286e-39, 1.3385246200397244e-48, 5.030576536912484e-50, 4.5788097680662457e-54, 5.532149677498453e-56, 7.883324070484006e-63]


In [1051]:
k = 50
cunt = 0
als = []
for i in range(len(data[k])):
    if preds[i] != data[k][i][1]:
        cunt = cunt + 1
        als.append(i)
for i in range(len(als)):
    print(data[k][als[i]])
    print(preds[als[i]])
cunt

('starts', 'VB')
NN


1

In [980]:
if "featured" in vocab:
    print(True)
else:
    print(False)

True


In [914]:
my_list = []
for i in range(len(all_predictions)):
    my_listt = []
    for j in range(len(all_predictions[i])):
        my_listt.append(tuple([test_data[i][j] , all_predictions[i][j]]))
    my_list.append(my_listt)

In [915]:
predictions = pd.DataFrame(columns = ["id" , "tagged_sentence"])
predictions['id'] = test['id']
predictions['tagged_sentence'] = my_list
predictions

,id,tagged_sentence
0,3,"[(For, NN), (you, PP), (have, HV), (been, BE),..."
1,4,"[(Man, NN), (had, HV), (enough, AP), (to, TO),..."
2,6,"[(He, NN), (trudged, VB), (on, RP), (,, ,), (h..."
3,12,"[(``, NN), (But, CC), (I, PP), (believe, VB), ..."
4,15,"[(then, NN), (,, ,), (to, IN), (better, JJ), (..."
...,...,...
3995,9990,"[(The, NN), (way, NN), (MacArthur, NN), (said,..."
3996,9991,"[(urged, NN), (the, NN), (reconvention, NN), (..."
3997,9994,"[(A, NN), (prospective, JJ), (industry, NN), (..."
3998,9997,"[(Mr., NN), (McCormack, NN), (., .)]"


In [916]:
predictions.to_csv("submission_6.csv" , index = False)